In [6]:
from pvd_io import *
import pickle

In [10]:
data_dir = 'pvd_analysis/'
min_file_size = 1e3  # 1 kb
datasets, sessions, files = scan_directories(data_dir, min_file_size, filetype='.pkl')
print(f"Located {len(files)} matched segment lists.")

Located 95 matched segment lists.


In [11]:
matched_segments_vx = []

for ii, session in enumerate(sessions):
    file_path = f"{data_dir}/{datasets[ii]}/{session}/length_change_voxels.pkl"
    with open(file_path, 'rb') as file:
        session_segments_vx = pickle.load(file)

    matched_segments_vx.append(session_segments_vx)

In [13]:
len(matched_segments_vx[1])

9

In [5]:
files

['length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_raw.pkl',
 'length_change_spline.pkl',
 'length_change_voxels.pkl',
 'length_change_spline.pkl',
 'length_change_v